# Solving Bike Repositioning problem as a Travelling Salesman Problem with Google OR-Tools

From the ouput of the minimum cost flow solver we have a list of edges between supply and demand nodes that need to be traversed. 

As a starting point for solving the bike repositioning problem we want to figure out the route of minimum cost that a single vehicle could take that traverses all these edges. This type of problem is referred to as the Travelling Salesmen Problem. In particular the travelling salesman problem states the following ["Given a list of cities and the distances between each pair of cities what is the shortest possible route that visits each city exactly once and returns to the origin city."] (https://en.wikipedia.org/wiki/Travelling_salesman_problem). 


Notice that this does not exactly match the output of our MCF solver. We do have a list of nodes and each one of these need to be visited. However we also have an additional constraint that the edges that have flow on them must be traversed. We could formulate the problem entirely as a seperate optimization problem but we would end up having to introudce multiple additional variables and the problem would probably become intractable. 

Instead we will introuduce a novel method for transforming the output of the Minimum Cost Flow problem into a format that can be solved by a generic TSP solver. 

## Network Transformation  

We want to convert our network our network to format such that visting each of the nodes in the network irrespective of order corresponds to satisfying all demand and supply in the network. We have from the MCF solver output a list of edges between supply and demand nodes that must be traversed in order to satisfy demand. 


In order to do this we will apply the following transformation 

* Convert each edge outgoing a supply node to a demand node into a new combined node 
* Between this new node and all other nodes in the network make the distance the original distance of the supply-demand edge, plus the cost of going from the demand node in the new node and the supply node and in the other node. 

The diagram below illustrates this. 

![Illustration](./Transformation.png)

On the new network moving from node $(s_1,d_1)$ to $(s_2,d_2)$ corresponds to going from node $s_1$ to $d_1$ to $s_2$ in the original network. So leaving a node on the new network corresponds to satisfying the demand on that node. Therefore if we leave every node in the network once we will have satisfied the demand. So if we solve the problem as a TSP and then instead of returning to the original node simply add the distance of $S_n$ to $D_n$ for the last node we will have solved the problem at a minimum cost for a single vehicle.



## Solving TSP using Google OR tools 

OR-Tools are an open source optimization tool developed by google. We will use their vehicle routing solver to solve the problem on our new network 

The basic format of the solver was taken from [a open source example](https://github.com/google/or-tools/blob/stable/ortools/constraint_solver/samples/tsp.py) but we have made some changes to the distance function and solution function. 

Before we define the solver lets collect the data from the MCF output and format it appropriately. We first read the stationEdges DataFrame and the model output key edges dictionary (edges with flow).

In [2]:
import pickle 
import pandas as pd
import os 


with open(os.path.join(os.getcwd(),r'pickles\keyEdges.pkl'), 'rb') as fp:
    keyEdges = pickle.load(fp)
stationEdges = pd.read_pickle(os.path.join(os.getcwd(),r'pickles\stationEdges.pkl'))


Key Edges are the edges that have non zero flow and are not source/sink connected edges. 
These then becoming the nodes in our new network.  

Key Edges is a  dictionary in the form $(s_n,d_n)$ : $w$ where $s_n$ is the supply node and $d_n$ is the demand node.$w$ is the length between $s_n$ and $d_n$.


Next we capture the distances between all supply and demand nodes in the **original network** along with their distances as a dictionary in the same form as the node network. Note that although the format of these are the same, the meaning is completly different in this context. The edges will be used to differently in the distance calculation. 

In [3]:
edgesDict = {}
for x in stationEdges.iterrows():
    edgesDict.update({ (str(x[0][0]) ,str(x[0][1])):int(round(x[1]['weight']))})


Finally lets define the google OR Model. 

In [6]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import jdc 
class TSPORSolver:

    def __init__(self,nodeDict,edges):
        self.nodeDict =  nodeDict
        self.edges = edges

    def create_data_model(self):
        data = {}
        data['locations'] = [x for x in self.nodeDict.keys()]
        self.locations = data['locations']
        data['num_vehicles'] = 1
        data['depot'] = 0
        return data
    def create_distance_callback(self,data, manager):
        distances_ = {}
        index_manager_ = manager
        for from_counter, from_node in enumerate(data['locations']):
            distances_[from_counter] = {}
            for to_counter, to_node in enumerate(data['locations']):
                if from_counter == to_counter:
                    distances_[from_counter][to_counter] = 0
                else:
                    distances_[from_counter][to_counter] = self.nodeDict.get(from_node) + self.edges.get((to_node[0],from_node[1]))           
        def distance_callback(from_index, to_index):
            from_node = index_manager_.IndexToNode(from_index)
            to_node = index_manager_.IndexToNode(to_index)
            return distances_[from_node][to_node]
        return distance_callback

    def print_solution(self,manager, routing, assignment):
            """Prints assignment on console."""
            print('Objective: {}'.format(assignment.ObjectiveValue()))
            index = routing.Start(0)
            plan_output = 'Route for vehicle 0:\n'
            route_distance = 0
            route_array = [] 
            while not routing.IsEnd(index):
                s = self.locations[index][0] 
                d = self.locations[index][1] 
                route_array.append(s)
                route_array.append(d)
                plan_output += ' {} ->'.format(s)
                plan_output += '{} ->'.format(d)
                previous_index = index
                index = assignment.Value(routing.NextVar(index))
                previous_arc_dist = routing.GetArcCostForVehicle(previous_index, index, 0)
                route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
            route_distance -= previous_arc_dist + self.edges.get((self.locations[previous_index][0],self.locations[previous_index][1]))
            plan_output += 'Distance of the route: {}m\n'.format(route_distance)
            print(plan_output)
          
    def solve(self):
        data = self.create_data_model()
        manager = pywrapcp.RoutingIndexManager(len(data['locations']),
                                            data['num_vehicles'], data['depot'])
        routing = pywrapcp.RoutingModel(manager)

        distance_callback = self.create_distance_callback(data, manager)
        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

        assignment = routing.SolveWithParameters(search_parameters)

        if assignment:
           return self.print_solution(manager, routing, assignment)

We create a distance callback function. This distance function differs significantly from the examples provided by google. We will consider the design of our new network when computing the distance. 

If the edge connects to itself then we set the distance to 0. 

Otherwise we set the value to be the distance as $distance(s_1,d_1) + distance(d_1,s_2)$ where $(s_1,d_1)$ is the node we are leaving from and $(s_2,d_n) $ is the node it is going too . We use the edges section that we previously input to get the distance $d_1 \rightarrow s_2 $ we use the nodeDict values to get $s_1 \rightarrow d_1.$

Next we define the print function. Instead of printing the route back to the original node, we set the route to end at the second last node. Remove the distance from the second last to the final node and add the distance from the second last node to its demand node. 

Next lets solve the model. 

In [7]:
solverTSP = TSPORSolver(keyEdges,edgesDict)
solverTSP.solve()

Objective: 1222964
Route for vehicle 0:
 84 ->17 -> 594 ->123 -> 72 ->123 -> 48 ->203 -> 48 ->136 -> 48 ->557 -> 71 ->365 -> 112 ->123 -> 338 ->170 -> 338 ->1 -> 809 ->1 -> 372 ->22 -> 335 ->68 -> 388 ->358 -> 64 ->83 -> 64 ->180 -> 64 ->169 -> 349 ->169 -> 129 ->169 -> 129 ->330 -> 192 ->106 -> 260 ->121 -> 383 ->43 -> 6 ->581 -> 6 ->208 -> 109 ->81 -> 109 ->315 -> 28 ->315 -> 28 ->76 -> 357 ->540 -> 69 ->540 -> 381 ->540 -> 13 ->540 -> 20 ->540 -> 69 ->795 -> 12 ->214 -> 797 ->131 -> 34 ->131 -> 25 ->131 -> 572 ->604 -> 572 ->131 -> 572 ->90 -> 456 ->90 -> 456 ->535 -> 456 ->590 -> 343 ->271 -> 343 ->312 -> 110 ->8 -> 456 ->247 -> 456 ->7 -> 545 ->457 -> 545 ->7 -> 545 ->713 -> 572 ->362 -> 462 ->362 -> 806 ->674 -> 804 ->674 -> 431 ->674 -> 431 ->439 -> 431 ->93 -> 234 ->695 -> 234 ->93 -> 798 ->339 -> 70 ->339 -> 4 ->227 -> 793 ->227 -> 162 ->796 -> 12 ->287 -> 12 ->796 -> 88 ->98 -> 65 ->98 -> 109 ->257 -> 6 ->43 -> 383 ->121 -> 260 ->210 -> 116 ->141 -> 116 ->370 -> 396 ->370 -> 

While this is an interesting approach, the way we have transformed the problem is limited. In particular it is difficult to solve the problem with multiple vehicles. So we will need to introduce another, albeit less efficient method to solve the problem. 